# Data Pre-precossing

In [2]:
# Library imports
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [3]:
# Set the base path for input data files
CURRENT_DIR = Path().resolve()
DATA_BASE_PATH = CURRENT_DIR.parent / "data"

In [4]:
# set the path for the cleaned data file
output_dir = os.path.join("..", "outputs", "data_output")
os.makedirs(output_dir, exist_ok=True)

- Combine all input data into one dataframe from different themes and regions
- add Region and Theme columns

In [5]:
# Collect all Excel files from subdirectories and combine them into a single DataFrame
# Add Region and Theme columns based on directory and file names
all_dfs = []

for region_dir in DATA_BASE_PATH.iterdir():
    if not region_dir.is_dir():
        continue

    for excel_file in region_dir.glob("*.xlsx"):
        df = pd.read_excel(excel_file)
        region = region_dir.name
        theme = excel_file.stem

        if "Country" in df.columns:
            idx = df.columns.get_loc("Country") + 1
            df.insert(idx, "Region", region)
            df.insert(idx + 1, "Theme", theme)
        else:
            df["Region"] = region
            df["Theme"] = theme
        all_dfs.append(df)

combined_df = pd.concat(all_dfs, ignore_index=True)

- Based on UN Development Coordination Office's target to JWP, First we need to exclude unnecessary feature prior to data processing

In [6]:
# Columns to keep
columns_to_keep = [
    "Country", "Region", "Theme", "Plan name", "Strategic priority code", "Strategic priority", "Outcome code", "Outcome", "Output code", 
    "Output", "Sub-Output code", "Sub-Output",
    "Start date", "End date", "Status", "Agencies", "SDG Targets", "SDG Goals",
    "Total required resources", "Total available resources", "Total expenditure resources"
]
for year in range(2016, 2029):
    columns_to_keep += [
        f"{year} Required", f"{year} Available", f"{year} Expenditure"
    ]
#  Retained columns for processing
df_drop = combined_df[[col for col in columns_to_keep if col in combined_df.columns]]

In [7]:
# save retained DataFrame to CSV
# df_drop.to_csv(CURRENT_DIR / "drop_data.csv", index=False)

Existing features for data cleaning
- Country
- Region
- Theme	
- Plan name	
- Strategic priority code	
- Strategic priority	
- Outcome code
- Outcome	
- Output code	
- Output
- Sub-Output code	
- Sub-Output
- Start date	
- End date	
- Status	
- Agencies	
- SDG Targets	
- SDG Goals	
- Gender marker	
- Human rights marker
- Total required
- Total available
- Total expenditure
- Financial features from 2016 to 2028

In [8]:
df_drop.shape

(17319, 60)

# Data Cleaning
### 1. Standardized country names

In [9]:
# Manual country normalization map
countries = {
    "Lao PDR": "Lao People's Democratic Republic",
    "Cote d'Ivoire": "Côte d’Ivoire",
    "Turkiye": "Turkey",
    "Viet Nam": "Vietnam",
    "the Caribbean": "Caribbean (region)",
    "Pacific": "Pacific (region)",
    "Democratic Republic of the Congo": "Congo, The Democratic Republic of the",
    "Palestine": "Palestine, State of",
    "Kosovo": "Kosovo"
}

# Apply normalization for country names
df_drop["Country"] = df_drop["Country"].replace(countries)
df_drop.loc[:, "Country"] = df_drop["Country"].str.strip()

/var/folders/yd/0rx8hpn96fd0480f9ndhwn_00000gn/T/ipykernel_53347/1832222513.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop["Country"] = df_drop["Country"].replace(countries)


- Use standardized country library
- helpful for later classification and dashboard

In [10]:
import pycountry

def validate_country(name):
    try:
        return pycountry.countries.lookup(name).name
    except LookupError:
        return name
 
df_drop.loc[:, "Country"] = df_drop["Country"].apply(validate_country)

- standardize all unified countries' name
- Now all the countries could be recognized by pycountry and unified for later analysis
- Since Pacific (region) and Caribbean (region) are provided only with region names, there is no specific name to be assigned

In [11]:
print("Cleaned Countries Sample:")
print(df_drop["Country"].dropna().unique()[:180])

Cleaned Countries Sample:
['Timor-Leste' 'Pakistan' 'Cambodia' 'Pacific (region)' 'Malaysia'
 'Philippines' "Lao People's Democratic Republic" 'Mongolia' 'Indonesia'
 'Viet Nam' 'Sri Lanka' 'India' 'China' 'Thailand' 'Papua New Guinea'
 'Nepal' 'Afghanistan' 'Bangladesh' 'Maldives' 'Bhutan'
 'Iran, Islamic Republic of' 'Myanmar' 'Kosovo' 'Albania' 'Uzbekistan'
 'Ukraine' 'Tajikistan' 'Moldova, Republic of' 'Turkey' 'Kazakhstan'
 'Kyrgyzstan' 'Bosnia and Herzegovina' 'Azerbaijan' 'Serbia' 'Montenegro'
 'Belarus' 'Georgia' 'North Macedonia' 'Turkmenistan' 'Armenia'
 'Palestine, State of' 'Syrian Arab Republic' 'Bahrain' 'Iraq' 'Jordan'
 'Yemen' 'Saudi Arabia' 'Lebanon' 'Caribbean (region)' 'Brazil'
 'Dominican Republic' 'Guatemala' 'El Salvador' 'Costa Rica' 'Mexico'
 'Colombia' 'Uruguay' 'Chile' 'Cuba' 'Honduras' 'Ecuador' 'Argentina'
 'Paraguay' 'Bolivia, Plurinational State of'
 'Venezuela, Bolivarian Republic of' 'Peru' 'Panama' 'Haiti' 'Zimbabwe'
 'Somalia' 'Sudan' 'South Africa' 'N

In [12]:
# check the shape with all countries
df_drop.shape

(17319, 60)

- Drop Regions 
1. Pacific (region)
2.	Caribbean (region)

In [13]:
# Define non-country entries and countries to drop
non_countries = ["Pacific (region)", "Caribbean (region)", "Kosovo"]

# Drop rows where 'Country' matches any in the non-country list or Kosovo
df_drop = df_drop[~df_drop["Country"].isin(non_countries)]
df_drop.shape

(15327, 60)

In [14]:
# Encode "Côte d’Ivoire" as "Ivory Coast" in the dataset using df_drop
df_drop["Country"] = df_drop["Country"].replace("Côte d’Ivoire", "Ivory Coast")

- Create another feature with pycountry lib for later country mapping convention (iso3)

In [15]:
def country_to_iso3_safe(name):
    # Handle special cases not recognized by pycountry
    if name == "Ivory Coast":
        return "CIV"
    if name == "Turkey":
        return "TUR"
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

df_drop.loc[:, "Country_ISO3"] = df_drop["Country"].apply(country_to_iso3_safe)

In [16]:
# Reorder columns to place Country_ISO3 after Country
country_index = df_drop.columns.get_loc("Country")
iso_column = df_drop.pop("Country_ISO3")
df_drop.insert(country_index + 1, "Country_ISO3", iso_column)
df_drop.head(2)

,Country,Country_ISO3,Region,Theme,Plan name,Strategic priority code,Strategic priority,Outcome code,Outcome,Output code,...,2025 Expenditure,2026 Required,2026 Available,2026 Expenditure,2027 Required,2027 Available,2027 Expenditure,2028 Required,2028 Available,2028 Expenditure
0,Timor-Leste,TLS,Asia Pacific,environment,United Nations Sustainable Development Coopera...,2.0,SUSTAINABLE ECONOMIC OPPORTUNITIES AND DECENT ...,2.0,"By 2025, institutions and people throughout Ti...",2.1 (Sub-outcome),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pakistan,PAK,Asia Pacific,environment,UNSDCF 2023 - 2027,NaN,Climate Change and the Environment,3.0,"Outcome 3: By 2027, people living in the Indus...",3.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Standardize JWP Plan names for each country under themes

- Normalization and basic cleaning for following categories
1. United Nations Sustainable Development Cooperation Framework
2. United Nations Development Assistance Framework

In [17]:
def clean_plan_name(text):
    if pd.isna(text):
        return None
    
    # Normalize text
    text = (
        text.replace("‚Äôs", "'s")
            .replace("â€“", "-")
            .replace("–", "-")
            .replace("‚Äì", "-")
            .replace("Ã", " ")
            .replace("√©", "é")
            .replace("√â", "à")
            .replace("√", "")
            .strip()
    )

    lower_text = text.lower()

    # Map to standardized names
    if "cooperation framework" in lower_text or "unsdcf" in lower_text:
        return "United Nations Sustainable Development Cooperation Framework"
    elif "undaf" in lower_text or "development assistance framework" in lower_text:
        return "United Nations Development Assistance Framework"
    else:
        return text

# Apply to column
df_drop.loc[:, "Plan name"] = df_drop["Plan name"].apply(clean_plan_name)

# Check unique standardized results
df_drop["Plan name"].value_counts()

Plan name
United Nations Sustainable Development Cooperation Framework                                                                               10204
United Nations Development Assistance Framework                                                                                             2544
Republic of Moldova - United Nations Partnership Framework for Sustainable Development  2018-2022                                            292
UNDAP 2016-2022                                                                                                                              188
UNSDF 2018 - 2022                                                                                                                            181
Viet Nam One Strategic Plan 2017-2021 (OSP)                                                                                                  157
UNPF 2018-2022                                                                                                          

#### Categorization
1. Sustainable Cooperation Framework:
- Republic of Moldova - United Nations Partnership Framework for Sustainable Development 2018-2022
- 2017-2021 MSDF
- UNSDF 2018 - 2022
- UNCF 2019 - 2022
- United Nations Sustainable Development Partnership Framework (2018-2022)
- Turkmenistan - United Nations Partnership Framework for Development 2016-2020
- United Nations Sustainable Development Framework (UNSDF) 2018-2022
- UNPF 2018-2022, UNPF 2017-2021
- Viet Nam One Strategic Plan 2017-2021 (OSP)
- United Nations Partnership Framework 2019-2024
- United Nations Sustainable Development Partnership
- Viet Nam's One Strategic Framework for Sustainable Development Cooperation 2022-2026
- UN Sudan Common Approach Result Framework 2024 to 2025

2. Development Assistance Framework:
- UNDAP 2016-2022
- United Nations Transitional Framework
- Transitional Engagement Framework (TEF)
- UNPDF 2016-2020
- PLAN CADRE DES NATIONS UNIES POUR L'ASSISTANCE AU DEVELOPPEMENT (PNUAD 2019-2023)
- Marco de Asistencia de las Naciones Unidas para el Desarrollo (MANUD)
- Cadre Programmatique Unique des Nations Unies pour l'assistance au Développement 2017-2020

In [18]:
from unidecode import unidecode

def standardize_plan_name(plan):
    if pd.isna(plan):
        return None

    # Normalize: remove accents, lower, strip, and simplify whitespace
    plan = unidecode(plan.lower().strip())
    plan = " ".join(plan.split())

    # === SD Cooperation Framework Keywords ===
    coop_keywords = [
        "cooperation framework", "sustainable development", "unsdcf", "unsdf",
        "uncf", "unpf", "msdf", "osp", "partnership framework",
        "framework for sustainable development", "cadre de cooperation", 
        "cadre de partenariat", "plan-cadre", "plan cadre de cooperation", 
        "marco de cooperacion", "plan marco", "cadre de developpement durable", 
        "cadre de cooperation pour le developpement durable", 
        "cadre de cooperation", "manud", "plan cadre de cooperation", "cadre de cooparation", 
        "marco de cooperacin", "marco de cooperaci", "marco de cooperacion", "desarrollo sostenible", 
        "plan marco", "cooperacion para el desarrollo"
    ]

    if any(k in plan for k in coop_keywords):
        return "United Nations Sustainable Development Cooperation Framework"

    # === Development Assistance Framework Keywords ===
    daf_keywords = [
        "undaf", "undap", "transitional", "strategic framework", 
        "un socio-economic", "resilience response", "peacebuilding", 
        "development assistance", "plan d'aide", "pdf", "common approach", 
        "pnuad", "cadre programmatique unique", "marco de asistencia", 
        "assistance au developpement", "aide au developpement", "plan d'aide", 
        "pnuad", "manud"
    ]
    
    if any(k in plan for k in daf_keywords):
        return "United Nations Development Assistance Framework"

    return plan

# Apply to DataFrame
df_drop["Plan name"] = df_drop["Plan name"].apply(standardize_plan_name)
df_drop["Plan name"].value_counts()

Plan name
United Nations Sustainable Development Cooperation Framework    11965
United Nations Development Assistance Framework                  3362
Name: count, dtype: int64

- Now, all noise on plan names are standardized into Cooperation Framework and Assistant Framework
- create a new variable (df_cp) as marking from cleaning process on country and plan features

In [19]:
df_cp = df_drop.copy()
df_cp.shape

(15327, 61)

In [20]:
df_cp.head(2)

,Country,Country_ISO3,Region,Theme,Plan name,Strategic priority code,Strategic priority,Outcome code,Outcome,Output code,...,2025 Expenditure,2026 Required,2026 Available,2026 Expenditure,2027 Required,2027 Available,2027 Expenditure,2028 Required,2028 Available,2028 Expenditure
0,Timor-Leste,TLS,Asia Pacific,environment,United Nations Sustainable Development Coopera...,2.0,SUSTAINABLE ECONOMIC OPPORTUNITIES AND DECENT ...,2.0,"By 2025, institutions and people throughout Ti...",2.1 (Sub-outcome),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pakistan,PAK,Asia Pacific,environment,United Nations Sustainable Development Coopera...,NaN,Climate Change and the Environment,3.0,"Outcome 3: By 2027, people living in the Indus...",3.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# df_cp.to_csv(CURRENT_DIR / "country_plan_cleaning.csv", index=False)

## 3. Cleaning Strategic Priority

- Checking priority code and normalize them
- it ranges from 1 to 10

In [22]:
# Count occurrences of each unique value in "Strategic priority code"
spc_counts = df_cp["Strategic priority code"].value_counts(dropna=False).reset_index()
spc_counts.columns = ["Strategic priority code", "Count"]
spc_counts

,Strategic priority code,Count
0,1.0,5053
1,2.0,4321
2,3.0,3548
3,4.0,1780
4,5.0,265
5,NaN,90
6,6.0,74
7,-,68
8,7.0,43
9,3.2,29


In [23]:
# Define a cleaning function for strategic priority codes
def clean_spc(value):
    if pd.isna(value) or value == '-':
        return np.nan
    if isinstance(value, str) and value.strip().startswith("SP"):
        try:
            return int(value.strip().split()[-1])
        except:
            return np.nan
    try:
        return int(float(value))
    except:
        return np.nan

# Apply the cleaning function
df_cp["Strategic priority code"] = df_cp["Strategic priority code"].apply(clean_spc)

In [24]:
# check value counts after cleaning
spc_counts_cleaned = df_cp["Strategic priority code"].value_counts(dropna=False).reset_index()
spc_counts_cleaned.columns = ["Strategic priority code", "Count"]
spc_counts_cleaned

,Strategic priority code,Count
0,1.0,5056
1,2.0,4326
2,3.0,3585
3,4.0,1780
4,5.0,265
5,NaN,158
6,6.0,74
7,7.0,43
8,9.0,18
9,8.0,16


- Filling missing data based on other features
1. outcome code
2. output code
3. sub-output code
- If the code is missing in strategic priority, we can refer those features

In [25]:
# Filter rows where SPC is NaN and select relevant columns
spc_missing = df_cp[df_cp["Strategic priority code"].isna()][[
    "Strategic priority code",
    "Outcome code",
    "Output code",
    "Sub-Output code"
]]
spc_missing.head(2)

,Strategic priority code,Outcome code,Output code,Sub-Output code
1,NaN,3.0,3.2,1
39,NaN,1.0,1.1,1.1.4


- Impute the missing data by referring 1st value before "."
1. 1st tier: outcome code
2. 2nd tier: output code
3. 3rd tier: sub-output code

In [26]:
# Impute NaN in SPC using Outcome code
def impute_spc(row):
    spc = row["Strategic priority code"]
    outcome = row["Outcome code"]
    output = row["Output code"]
    sub_output = row["Sub-Output code"]

    if pd.isna(spc):
        # First try Outcome code
        try:
            outcome_clean = int(float(outcome))
            return outcome_clean
        except:
            pass

        # If Outcome is missing or invalid, try Output code
        try:
            if isinstance(output, str) and '.' in output:
                first_digit = int(output.strip().split('.')[0])
                return first_digit
        except:
            pass

        # If Output is missing or invalid, try Sub-Output code
        try:
            if isinstance(sub_output, str) and '.' in sub_output:
                first_digit = int(sub_output.strip().split('.')[0])
                return first_digit
        except:
            pass

    return spc  # Return original if no imputation needed

# Apply imputation
df_cp["Strategic priority code"] = df_cp.apply(impute_spc, axis=1)

# Check the counts after imputation
spc_counts_imputed = df_cp["Strategic priority code"].value_counts(dropna=False).reset_index()
spc_counts_imputed.columns = ["Strategic priority code", "Count"]
spc_counts_imputed

,Strategic priority code,Count
0,1.0,5117
1,2.0,4349
2,3.0,3607
3,4.0,1824
4,5.0,273
5,6.0,74
6,7.0,43
7,9.0,18
8,8.0,16
9,10.0,6


- Create country-theme-priority specific code
- each country has its own definition usage of priority code
- helpful for later combination of sub-output under same priority

In [27]:
# Create a standardized label: country_theme_spc + original sp text
df_cp["SP_Label"] = (
    df_cp["Country_ISO3"].str.replace(" ", "") + "_" +
    df_cp["Theme"].str.lower().str.replace(" ", "").str.replace("/", "_") + "_" +
    df_cp["Strategic priority code"].astype(str).str.replace(".0", "", regex=False)
)

# Reorder columns
cols = list(df_cp.columns)
plan_idx = cols.index("Plan name")
cols.remove("SP_Label")
cols.insert(plan_idx + 1, "SP_Label")
df_cp = df_cp[cols]
df_cp.head(2)

,Country,Country_ISO3,Region,Theme,Plan name,SP_Label,Strategic priority code,Strategic priority,Outcome code,Outcome,...,2025 Expenditure,2026 Required,2026 Available,2026 Expenditure,2027 Required,2027 Available,2027 Expenditure,2028 Required,2028 Available,2028 Expenditure
0,Timor-Leste,TLS,Asia Pacific,environment,United Nations Sustainable Development Coopera...,TLS_environment_2,2.0,SUSTAINABLE ECONOMIC OPPORTUNITIES AND DECENT ...,2.0,"By 2025, institutions and people throughout Ti...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pakistan,PAK,Asia Pacific,environment,United Nations Sustainable Development Coopera...,PAK_environment_3,3.0,Climate Change and the Environment,3.0,"Outcome 3: By 2027, people living in the Indus...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Remove the features

In [28]:
# Drop specified columns from df_cp
columns_to_drop = [
    "Strategic priority", "Outcome code", "Outcome", "Output code", "Output", "Sub-Output code", "Start date",
    "End date", "Status"
]
df_cp = df_cp.drop(columns=columns_to_drop)

df_cp.head(2)

,Country,Country_ISO3,Region,Theme,Plan name,SP_Label,Strategic priority code,Sub-Output,Agencies,SDG Targets,...,2025 Expenditure,2026 Required,2026 Available,2026 Expenditure,2027 Required,2027 Available,2027 Expenditure,2028 Required,2028 Available,2028 Expenditure
0,Timor-Leste,TLS,Asia Pacific,environment,United Nations Sustainable Development Coopera...,TLS_environment_2,2.0,2.1.1 Rural socio-economic development through...,International Labour Organisation; UN Women; U...,"1.1 By 2030, eradicate extreme poverty for all...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pakistan,PAK,Asia Pacific,environment,United Nations Sustainable Development Coopera...,PAK_environment_3,3.0,Capacity development for an inclusive science...,United Nations Environment Programme,"12.2 By 2030, achieve the sustainable manageme...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_cp.shape

(15327, 53)

In [30]:
df_sp = df_cp.copy()

In [31]:
# save retained DataFrame to CSV
# df_sp.to_csv(CURRENT_DIR / "SP Cleaning.csv", index=False)

## 4. Dataset Splitting

- Create 3 database for different usage
1. SDG Goals collection
2. UN Agencies targets
3. Financial support w.r.t Strategic Priority

In [32]:
# 1. Database with UN Agencies targets
db1 = df_sp[[
    "Country", "Country_ISO3", "Region", "Theme", "Plan name", "SP_Label",
    "Strategic priority code", "Agencies"
]]

In [33]:
# 2. Database with SDG Goals
db2 = df_sp[[
    "Country", "Country_ISO3", "Region", "Theme", "Plan name", "SP_Label",
    "Strategic priority code", "SDG Goals"
]]

In [34]:
# 3. Database merged by SP_Label, sum financials, aggregate text columns

# List of columns in the desired order
columns_order = [
    "Country", "Country_ISO3", "Region", "Theme", "Plan name", "SP_Label", "Strategic priority code",
    "Sub-Output", "Agencies", "SDG Targets", "SDG Goals",
    "Total required resources", "Total available resources", "Total expenditure resources"
]

# Add all year-based financial columns
for year in range(2016, 2029):
    columns_order += [
        f"{year} Required", f"{year} Available", f"{year} Expenditure"
    ]

# Build aggregation dictionary
agg_dict = {
    "Country": "first",
    "Country_ISO3": "first",
    "Region": "first",
    "Theme": "first",
    "Plan name": "first",
    "SP_Label": "first",
    "Strategic priority code": "first",
    "Sub-Output": lambda x: "; ".join(sorted(set(x.dropna()))),
    "Agencies": lambda x: "; ".join(sorted(set(x.dropna()))),
    "SDG Targets": lambda x: "; ".join(sorted(set(x.dropna()))),
    "SDG Goals": lambda x: "; ".join(sorted(set(x.dropna()))),
    "Total required resources": "sum",
    "Total available resources": "sum",
    "Total expenditure resources": "sum",
}

for year in range(2016, 2029):
    agg_dict[f"{year} Required"] = "sum"
    agg_dict[f"{year} Available"] = "sum"
    agg_dict[f"{year} Expenditure"] = "sum"

# Only keep columns that exist in your DataFrame
columns_to_use = [c for c in columns_order if c in df_cp.columns]

# Group and aggregate
db3 = df_cp[columns_to_use].groupby("SP_Label", as_index=False).agg(agg_dict)

# Reorder columns to match your desired order
db3 = db3[[c for c in columns_order if c in db3.columns]]

In [35]:
# Save to CSV or SQL as needed
db1.to_csv("UN_Agencies.csv", index=False)
db2.to_csv("SDG_Goals.csv", index=False)
db3.to_csv("Financial.csv", index=False)

## 5. SDG feature cleaning
- Splits each SDG Goals cell by "; "
- Extracts the SDG code and content for each goal
- Expands rows so each SDG goal is a separate row, duplicating other columns as needed

In [36]:
df_sdg = pd.read_csv("SDG_Goals.csv")

def split_sdg_goals(row):
    sdg_goals = str(row['SDG Goals'])
    if sdg_goals.strip() == "-" or pd.isna(sdg_goals):
        return []
    parts = [g.strip() for g in sdg_goals.split(";")]
    result = []
    for part in parts:
        if part and part[0].isdigit():
            code = part.split(" ", 1)[0]
            content = part[len(code):].strip()
            if content.endswith("- Strong Institutions"):
                content = content.replace("- Strong Institutions", "and Strong Institution").strip()
            result.append({
                **row.drop('SDG Goals').to_dict(),
                "SDG Code": code,
                "SDG Goals": content
            })
    return result

expanded = []
for _, row in df_sdg.iterrows():
    expanded += split_sdg_goals(row)

df_sdg_clean = pd.DataFrame(expanded)

# Remove duplicates for exact same rows
df_sdg_clean = df_sdg_clean.drop_duplicates()

# Remove duplicates in column selection
base_cols = [c for c in df_sdg_clean.columns if c not in ["SDG Goals", "SDG Code"]]
cols = base_cols + ["SDG Code", "SDG Goals"]
df_expanded = df_sdg_clean[cols]

# Save to new CSV
df_expanded.to_csv(os.path.join(output_dir, "SDG_Goals_Cleaned.csv"), index=False)

In [37]:
df_expanded.head(2)

,Country,Country_ISO3,Region,Theme,Plan name,SP_Label,Strategic priority code,SDG Code,SDG Goals
0,Timor-Leste,TLS,Asia Pacific,environment,United Nations Sustainable Development Coopera...,TLS_environment_2,2.0,1,No Poverty
1,Timor-Leste,TLS,Asia Pacific,environment,United Nations Sustainable Development Coopera...,TLS_environment_2,2.0,5,Gender Equality


## 6. UN Agencies Cleaning
- For each row, split the "Agencies" column by ";".
- For rows with multiple agencies, duplicate the row for each agency, keeping other columns the same.
- Strip whitespace from agency names.
- Output a DataFrame with the same columns, but each row has only one agency.

In [38]:
df_agency = pd.read_csv("UN_Agencies.csv")

def split_agencies(row):
    agencies = str(row['Agencies'])
    if pd.isna(agencies) or agencies.strip() == "-":
        return []
    parts = [a.strip() for a in agencies.split(";")]
    return [
        {**row.drop('Agencies').to_dict(), "Agencies": agency}
        for agency in parts if agency
    ]

expanded_agencies = []
for _, row in df_agency.iterrows():
    expanded_agencies += split_agencies(row)

df_agency_clean = pd.DataFrame(expanded_agencies)
df_agency_clean = df_agency_clean.drop_duplicates()
df_agency_clean = df_agency_clean[df_agency.columns]

# Save to new CSV
df_agency_clean.to_csv(os.path.join(output_dir, "UN_Agencies_Cleaned.csv"), index=False)

In [39]:
df_agency_clean.head(2)

,Country,Country_ISO3,Region,Theme,Plan name,SP_Label,Strategic priority code,Agencies
0,Timor-Leste,TLS,Asia Pacific,environment,United Nations Sustainable Development Coopera...,TLS_environment_2,2.0,International Labour Organisation
1,Timor-Leste,TLS,Asia Pacific,environment,United Nations Sustainable Development Coopera...,TLS_environment_2,2.0,UN Women


## 7. Financial Data Cleaning
- De-duplication of UN agencies
- re-arranging future financial data rolling

In [40]:
df_fin = pd.read_csv("Financial.csv")

def dedup_agencies(cell):
    if pd.isna(cell) or cell.strip() == "-":
        return cell
    agencies = [a.strip() for a in cell.split(";")]
    unique_agencies = sorted(set(a for a in agencies if a))
    return "; ".join(unique_agencies)

def dedup_sdg_goals(cell):
    if pd.isna(cell) or cell.strip() == "-":
        return cell
    goals = [g.strip() for g in cell.split(";")]
    cleaned_goals = []
    for g in goals:
        # Remove leading numbers and spaces
        cleaned = g.lstrip("0123456789. ").strip()
        # Remove "- Strong Institutions" if present
        if cleaned.endswith("- Strong Institutions"):
            cleaned = cleaned.replace("- Strong Institutions", "and Strong Institution").strip()
        if cleaned:
            cleaned_goals.append(cleaned)
    unique_goals = sorted(set(cleaned_goals))
    return "; ".join(unique_goals)

# De-duplicate
df_fin["Agencies"] = df_fin["Agencies"].apply(dedup_agencies)
df_fin["SDG Goals"] = df_fin["SDG Goals"].apply(dedup_sdg_goals)

# Expand so each row has only one agency and one SDG Goal
def expand_agency_sdg(row):
    agencies = [a.strip() for a in str(row['Agencies']).split(";") if a.strip() and a.strip() != "-"]
    sdgs = [g.strip() for g in str(row['SDG Goals']).split(";") if g.strip() and g.strip() != "-"]
    if not agencies:
        agencies = [""]
    if not sdgs:
        sdgs = [""]
    expanded = []
    for agency in agencies:
        for sdg in sdgs:
            new_row = row.copy()
            new_row['Agencies'] = agency
            new_row['SDG Goals'] = sdg
            expanded.append(new_row)
    return expanded

expanded_rows = []
for _, row in df_fin.iterrows():
    expanded_rows.extend(expand_agency_sdg(row))

df_fin_expanded = pd.DataFrame(expanded_rows)

# Group by SP_Label and aggregate
agg_dict = {
    "Country": "first",
    "Country_ISO3": "first",
    "Region": "first",
    "Theme": "first",
    "Plan name": "first",
    "SP_Label": "first",
    "Strategic priority code": "first",
    "Sub-Output": lambda x: "; ".join(sorted(set(x.dropna()))),
    "Agencies": lambda x: "; ".join(sorted(set(x.dropna()))),
    "SDG Targets": lambda x: "; ".join(sorted(set(x.dropna()))),
    "SDG Goals": lambda x: "; ".join(sorted(set(x.dropna()))),
    "Total required resources": "sum",
    "Total available resources": "sum",
    "Total expenditure resources": "sum",
}

# Add all year-based financial columns if present
for year in range(2016, 2029):
    for col_type in ["Required", "Available", "Expenditure"]:
        col = f"{year} {col_type}"
        if col in df_fin_expanded.columns:
            agg_dict[col] = "sum"

df_fin_grouped = df_fin_expanded.groupby("SP_Label", as_index=False).agg(agg_dict)

# Move 2026–2028 financials into 2025 columns if non-zero, then zero out 2026–2028
for col_type in ["Required", "Available", "Expenditure"]:
    for year in [2026, 2027, 2028]:
        src_col = f"{year} {col_type}"
        tgt_col = f"2025 {col_type}"
        if src_col in df_fin.columns and tgt_col in df_fin_grouped.columns:
            df_fin_grouped[tgt_col] = df_fin_grouped[tgt_col] + df_fin_grouped[src_col].fillna(0)
            df_fin_grouped[src_col] = 0

# Move 2016–2019 financials into 2020 columns if non-zero, then zero out 2016–2019
for col_type in ["Required", "Available", "Expenditure"]:
    tgt_col = f"2020 {col_type}"
    for year in [2016, 2017, 2018, 2019]:
        src_col = f"{year} {col_type}"
        if src_col in df_fin_grouped.columns and tgt_col in df_fin_grouped.columns:
            df_fin_grouped[tgt_col] = df_fin_grouped[tgt_col] + df_fin_grouped[src_col].fillna(0)
            df_fin_grouped[src_col] = 0

# Remove columns for 2016–2019 and 2027–2028 for Required, Available, and Expenditure
years_to_remove = list(range(2016, 2020)) + [2027, 2028]
col_types = ["Required", "Available", "Expenditure"]

cols_to_drop = []
for year in years_to_remove:
    for col_type in col_types:
        col = f"{year} {col_type}"
        if col in df_fin_grouped.columns:
            cols_to_drop.append(col)

df_fin_grouped = df_fin_grouped.drop(columns=cols_to_drop)

In [41]:
df_fin_grouped.to_csv(os.path.join(output_dir, "Financial_Cleaned.csv"), index=False)

In [42]:
df_fin_grouped.head(2)

,Country,Country_ISO3,Region,Theme,Plan name,SP_Label,Strategic priority code,Sub-Output,Agencies,SDG Targets,...,2023 Expenditure,2024 Required,2024 Available,2024 Expenditure,2025 Required,2025 Available,2025 Expenditure,2026 Required,2026 Available,2026 Expenditure
0,Afghanistan,AFG,Asia Pacific,crime,United Nations Development Assistance Framework,AFG_crime_3,3.0,3.2.3 Awareness raising on effective legal and...,United Nations Assistance Mission in Afghanistan,"10.2 By 2030, empower and promote the social, ...",...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0,0,0
1,Afghanistan,AFG,Asia Pacific,education,United Nations Development Assistance Framework,AFG_education_1,1.0,"""1.2.5 Support access to formal education (pri...",United Nations Children's Fund; United Nations...,"2.1 By 2030, end hunger and ensure access by a...",...,3.065378e+09,8.904746e+09,5.411128e+09,4.939757e+09,5.599188e+09,1.523980e+09,0.0,0,0,0
